<a href="https://colab.research.google.com/github/WilliamPoe/CSCI-290/blob/main/notebooks/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import math

In [32]:
def entropy(df, target, attribs):
  entropy = 0
  lowest_entropy = float('inf')
  overall = len( df )
  ## Remove the [2:] ##
  for attrib in attribs[2:]:
    vals = df[attrib].unique()
    for val in vals:
      subset_size = len(df[ df[attrib] == val ])
      weight = subset_size / overall
      props = df[ df[attrib] == val ][target].value_counts( normalize=True )
      for p in props.array:
        entropy =  entropy - weight*(p*math.log2(p))
    if lowest_entropy > entropy:
      lowest_entropy = entropy
      lowest_attrib = attrib
  return lowest_attrib, lowest_entropy

In [39]:
# possible check what the values are for that attribute and then call a function that does
# categorical or quantitative if it has more than 10 values
# categorical name and entropy
# quantitative name and splitting point

In [37]:
def gini_impurity(df, target, attribs):
  gini = 0
  lowest_gini = float('inf')
  overall = len( df )
  ## Remove the [2:] ##
  for attrib in attribs[2:]:
    vals = df[attrib].unique()
    for val in vals:
      subset_size = len(df[ df[attrib] == val ])
      weight = subset_size / overall
      props = df[ df[attrib] == val ][target].value_counts( normalize=True )
      for p in props.array:
        gini =  weight*(1-(p**2))
    if lowest_gini > gini:
      lowest_gini = gini
      lowest_attrib = attrib
  return lowest_attrib, lowest_gini

In [5]:
def main ():
  df = pd.read_csv("https://github.com/WilliamPoe/CSCI-290/raw/main/Data/ad_click_dataset.csv")
  attribs = []
  print(df.columns)
  target = str(input("What is your target column?: "))
  for col in df.columns:
    if col != target:
      attribs.append(col)
  measure = input("Would you like to use: \n1. Entropy \n2. Gini impurity \nSelection:")
  if measure == '1':
    print(entropy(df, target, attribs))
  else:
    print(gini_impurity(df, target, attribs))

In [38]:
main()

Index(['id', 'full_name', 'age', 'gender', 'device_type', 'ad_position',
       'browsing_history', 'time_of_day', 'click'],
      dtype='object')
What is your target column?: click
Would you like to use: 
1. Entropy 
2. Gini impurity 
Selection:2
('age', 0.005131578947368422)


In [28]:
main()

Index(['id', 'full_name', 'age', 'gender', 'device_type', 'ad_position',
       'browsing_history', 'time_of_day', 'click'],
      dtype='object')
What is your target column?: click
Would you like to use: 
1. Entropy 
2. Gini impurity 
Selection:1
('age', 0.4649645748457401)
